Рассматриваемый организм - Saccharomyces cerevisiae (пекарские дрожжи)

1) Найдем кол-во участков Z-ДНК в хромосоме chrXII

In [ ]:
!gcc zhunt3-alan.c -lm -o zhunt3
!chmod a+x zhunt3
!./zhunt3 12 8 12 chrXII.fa

In [46]:
import pandas as pd
table = pd.read_csv("chrXII.fa.Z-SCORE", skiprows=1, names=["Start","End","1","2","3","Score","Seq","4"], delim_whitespace=True)
table = table.loc[table['Score'] >= 300]
table

,Start,End,1,2,3,Score,Seq,4
1,2,26,24,21.989,58.056,413.7336,cacacacacacaccacccacacac,ASASASASASASSASASASASASA
49,50,74,24,22.151,57.615,344.1131,cacaccacacccacacacccacac,ASASSASASASASASASASASASA
50,51,75,24,21.703,59.310,577.1522,acaccacacccacacacccacaca,SAASASASASASASASASASASAS
51,52,76,24,21.677,59.749,595.3441,caccacacccacacacccacacac,ASSASASASASASASASASASASA
52,53,77,24,21.225,62.151,1029.4250,accacacccacacacccacacaca,ASASASASASASASASASASASAS
...,...,...,...,...,...,...,...,...
1078173,1078174,1078194,20,22.096,44.869,366.3665,tatttccacacacacacacc,ASASASASASASASASASAS
1078174,1078175,1078193,18,21.980,44.729,418.0957,atttccacacacacacac,SASASASASASASASASA
1078175,1078176,1078194,18,21.917,47.249,449.7606,tttccacacacacacacc,ASASASASASASASASAS
1078176,1078177,1078193,16,21.802,33.440,513.8668,ttccacacacacacac,SASASASASASASASA


Это участки Z-ДНК, в рассматриваемой хромосоме их 1431.

2) Посчитаем кол-во квадруплексов

In [ ]:
!pip install biopython
import re
from Bio import SeqIO

In [48]:
input_file = "chrXII.fa"
fasta_sequence = SeqIO.parse(input_file,'fasta')
for record in SeqIO.parse(input_file, "fasta"):
  print("%s %i" % (record.id, len(record)))
  name, sequence = record.id, str(record.seq)
pattern="(?:G{3,}[ATGC]{1,7}){3,}G{3,}"
PQS=[[m.start(),m.end(),m.group(0)] for m in re.finditer(pattern,sequence,re.IGNORECASE)]
len(PQS)

chrXII 1078177


3

В хромосоме есть 3 квадруплекса

3) Скачаем разметку генома - sacCer3.ensGene.gtf

In [ ]:
%%bash
wget https://github.com/arq5x/bedtools2/releases/download/v2.29.1/bedtools-2.29.1.tar.gz
tar -zxvf bedtools-2.29.1.tar.gz

In [ ]:
%%bash
cd bedtools2
make

In [49]:
with open('./quadrs.bed', 'w') as the_file:
    for i in range(len(PQS)):
      the_file.write("chrXII\t"+str(PQS[i][0])+"\t"+str(PQS[i][1])+"\n")  

In [50]:
!./bedtools2/bin/bedtools intersect -a quadrs.bed -b sacCer3.ensGene.gtf | wc -l

0


In [51]:
with open('./zdna.bed', 'w') as zdna_file:
  for ind, row in table.iterrows():
    zdna_file.write("chrXII\t" + str(row['Start']) + "\t" + str(row['End']) + "\n")

In [52]:
!./bedtools2/bin/bedtools intersect -a zdna.bed -b sacCer3.ensGene.gtf -wao | tail

chrXII	1073317	1073333	chrXII	ensGene.v101	CDS	1072508	1077895	.	+	0	gene_id "YLR467W"; transcript_id "YLR467W_mRNA"; exon_number "1"; exon_id "YLR467W_mRNA.1"; gene_name "YLR467W";	16
chrXII	1073319	1073335	chrXII	ensGene.v101	transcript	1072508	1077898	.	+	.	gene_id "YLR467W"; transcript_id "YLR467W_mRNA";  gene_name "YLR467W";	16
chrXII	1073319	1073335	chrXII	ensGene.v101	exon	1072508	1077898	.	+	.	gene_id "YLR467W"; transcript_id "YLR467W_mRNA"; exon_number "1"; exon_id "YLR467W_mRNA.1"; gene_name "YLR467W";	16
chrXII	1073319	1073335	chrXII	ensGene.v101	CDS	1072508	1077895	.	+	0	gene_id "YLR467W"; transcript_id "YLR467W_mRNA"; exon_number "1"; exon_id "YLR467W_mRNA.1"; gene_name "YLR467W";	16
chrXII	1078173	1078193	.	.	.	-1	-1	.	.	.	.	0
chrXII	1078174	1078194	.	.	.	-1	-1	.	.	.	.	0
chrXII	1078175	1078193	.	.	.	-1	-1	.	.	.	.	0
chrXII	1078176	1078194	.	.	.	-1	-1	.	.	.	.	0
chrXII	1078177	1078193	.	.	.	-1	-1	.	.	.	.	0
chrXII	1078178	1078194	.	.	.	-1	-1	.	.	.	.	0


Т.о. все квадруплексы лежат в межгенном пространстве, а Z-ДНК попадают внутрь генов, на границу и в межгенное пространство

4-5) Выделим все промоторы и посмотрим на пересечения

In [63]:
!grep -w "chrXII" sacCer3.ensGene.gtf | grep -w "start_codon" > starts.gtf
!awk '{if ($7 == "+") print $1, ($4 - 1000), $4, $18; else print $1, $5, ($5 + 1000), $18}' starts.gtf > promoters.txt
with open('promoters.txt', 'r') as pin:
  with open('promoters.gtf', 'w') as pout:
    for line in pin:
      chr, start, end, name = line.split()
      pout.write(chr + "\t" + str(max(0, int(start))) + "\t" + str(end) + "\t" + name[1:-2] + "\n")

In [80]:
def distinct_and_write(fin, fout):
  genes = set()
  with open(fin, 'r') as zin:
    for name in zin:
      genes.add(name.strip())
  genes.remove('.')
  with open(fout, 'w') as zout:
    for gene in genes:
      zout.write(gene + "\n")

In [84]:
!./bedtools2/bin/bedtools intersect -a zdna.bed -b promoters.gtf -wao | awk '{print $7}' > Z-intersect.txt
distinct_and_write('Z-intersect.txt', 'Z-genes.txt')
!head Z-genes.txt

YLR072W
YLR183C
YLL028W
YLR278C
YLR354C
YLR320W
YLR389C
YLR390W-A
YLL001W
YLR378C


In [87]:
!./bedtools2/bin/bedtools intersect -a quadrs.bed -b promoters.gtf -wao | awk '{print $7}' > Q-intersect.txt
distinct_and_write('Q-intersect.txt', 'Q-genes.txt')
!head Q-genes.txt

YLR462W
YLR312W-A
YLR312C


Т.о. в промоторы генов из Z-genes.txt попадают Z-ДНК, в промоторы генов из Q-genes.txt - квадруплексы